In [ ]:
%cd /content/

/content


In [ ]:
!git clone https://github.com/pgroupATusc/TransFetch

Cloning into 'TransFetch'...
remote: Enumerating objects: 202, done.
remote: Counting objects: 100% (202/202), done.
remote: Compressing objects: 100% (161/161), done.
remote: Total 202 (delta 50), reused 179 (delta 37), pack-reused 0
Receiving objects: 100% (202/202), 7.79 MiB | 6.27 MiB/s, done.
Resolving deltas: 100% (50/50), done.


In [ ]:
%cd /content/TransFetch/

/content/TransFetch


In [ ]:
!cp /content/drive/MyDrive/data/623.xalancbmk-s5.txt.xz .
!xz --decompress 623.xalancbmk-s5.txt.xz

In [ ]:
!cp /content/drive/MyDrive/data/trace-3rd-run-1M.out.tar.gz .
! tar -xvf trace-3rd-run-1M.out.tar.gz
!cp trace-3rd-run-1M.out in.txt

trace-3rd-run-1M.out


In [ ]:
!cp 623.xalancbmk-s5.txt testxalanc.txt

In [ ]:
!head -741262 trace-3rd-run-1M.out >test_temp.txt

In [ ]:
!wc -l testxalanc.txt test_temp.txt

  741261 testxalanc.txt
  741262 test_temp.txt
 1482523 total


In [ ]:
!sed '1d' test_temp.txt >test.txt

In [ ]:
import re
import csv
import random

infile='test.txt'
infile_xalanc= 'testxalanc.txt'

outfile='output.txt'
UID=[]
INST=[]
I=[]
# Read the data from the file
with open(infile, 'r') as file:
    data = file.readlines()
with open(infile_xalanc, 'r') as filexalanc:
    dataxalanc = filexalanc.readlines()
var=0
pattern = r'ts=(\d+)\s+pid=(\d+)\s+inst=(\d+)\s+cpuid=(\d+)\s+rip=([\da-fx]+)\s+va=([\da-fx]+)'
i=4
# Prepare the CSV data
csv_data = []
for line, linexalanc in zip(data, dataxalanc):
#for line in data:
    #print(line)
    #print(linexalanc)
    #match = re.match(r'ts=(\d+) mstress pid=(\d+)  inst=(\d+) cpuid=(\d+) rip=([a-f\d]+) va=([a-f\d]+)', line)
    match = re.match(pattern, line)
    instr_ptr=linexalanc.split(sep=',')[3]
    uid=linexalanc.split(sep=',')[0]
    UID.append(uid)
    I.append(i)
    #print(linexalanc.split()[3])
    if match:
        #print('match')
        ts, pid, inst, cpuid, rip, va = match.groups()

        INST.append(inst)
        #csv_data.append([ts, pid, inst, cpuid, rip, '0x'+va])
        #csv_data.append(['0x'+va+":",'R', '0x'+va])
        #csv_data.append([va+":",'R', va])
        # Convert the hexadecimal address to an integer
        addr_int = int(va, 16)

        # Shift the address right by 12 bits
        shifted_addr_int = addr_int >> 12

        # Convert the shifted integer address back to hexadecimal format
        shifted_addr_hex = hex(shifted_addr_int)
        #shifted_addr_hex=va
        #csv_data.append([ts, pid, inst, cpuid, rip, '0x'+va])
        #Format Unique Instr Id, Cycle Count, Load Address, Instruction Pointer of the Load, LLC hit/miss
        #sample 4, 15, 28e837c88340, 406a82, 0
        modified_shifted_addr_hex = shifted_addr_hex.replace('0x', '')
        rip= rip.replace('0x', '')
        #print("shifted_addr_hex",shifted_addr_hex,"modified_shifted_addr_hex", modified_shifted_addr_hex)
        #increment = random.randint(increment_min, increment_max)
        #current_value += increment
        #increment2 = random.randint(increment_min2, increment_max2)
        #current_value2 += increment2

        #csv_data.append([str(current_value2), ' '+str(current_value), ' '+str(modified_shifted_addr_hex), ' '+str(random.choice(values)), ' '+str(0)])
        #Unique Instr ID = inst, Cycle Count = ts, Load Address = va >>12, Intruction Pointer of Load = rip, LLC hit/miss = R/W
        #csv_data.append([str(inst), ' '+str(ts), ' '+str(modified_shifted_addr_hex), ' '+str(rip), ' '+str(0)])
        csv_data.append([str(uid), ' '+str(ts), ' '+str(modified_shifted_addr_hex), ' '+str(rip), ' '+str(0)])
        #csv_data.append([str(i), ' '+str(ts), ' '+str(modified_shifted_addr_hex), ' '+str(rip), ' '+str(0)])
        i=i+1

# Write CSV data to a file
with open(outfile, 'w', newline='') as csvfile:
    csv_writer = csv.writer(csvfile,delimiter=",")
    #csv_writer.writerow(['ts', 'pid', 'inst', 'cpuid', 'rip', 'va'])
    csv_writer.writerows(csv_data)
    print("csv_data",csv_data)


print("CSV conversion completed.")

csv_data 

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [ ]:
!pip install einops

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 1.2 MB/s eta 0:00:00


In [ ]:
!rm -rf ./res

In [ ]:
!./run_train.sh

TRAIN/VAL/TEST/SKIP: 2/1/2/0
testxalanc.txt
.//testxalanc.txt
train and validation
preprocessing with context
preprocessing with context
-------------Data Proccessed------------
Epoch: 1 - loss: 0.6136854854 - test_loss: 0.4747471511
-------- Save Best Model! --------
Epoch: 2 - loss: 0.4153436902 - test_loss: 0.3358076215
-------- Save Best Model! --------
Epoch: 3 - loss: 0.3120784743 - test_loss: 0.2473185509
-------- Save Best Model! --------
Epoch: 4 - loss: 0.2443815867 - test_loss: 0.1882347067
-------- Save Best Model! --------
Epoch: 5 - loss: 0.2005022723 - test_loss: 0.1489560952
-------- Save Best Model! --------
Epoch: 6 - loss: 0.1717816393 - test_loss: 0.1226063569
-------- Save Best Model! --------
Epoch: 7 - loss: 0.1528032530 - test_loss: 0.1043812086
-------- Save Best Model! --------
Epoch: 8 - loss: 0.1403050870 - test_loss: 0.0914771234
-------- Save Best Model! --------
Epoch: 9 - loss: 0.1322252448 - test_loss: 0.0822092195
-------- Save Best Model! --------
Epo